# HIV Model
Riya, Dhara, Chris

10-15-18

State: uninfected cells, latent cells, active cells, viron cells

System: rates

Parameters: initial number of infected cells

In [ ]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *


### Define State for the initial condition
- Uninfected Cells, Latent Cells, Active Cells, Viron Cells

In [ ]:
init = State(uninf = 1000, lat = 0, act = 0, vir = 100)

### Define System (Rates)

In [ ]:
gammat = 1.36*0.2 # Tau * Gamma

system = System ( gammat = gammat, 
                  sigma = 2, 
                  beta = 0.00027, 
                  alpha = 0.036, 
                  mu = .00136, 
                  pi = 100, 
                  delta = 0.33,
                  rho = 0.1,
                  t0 = 0, 
                  t_end = 1200)

## Update Function

In [ ]:
def update_func2(state, t, system): 
    
    
    #FLOW IN
    uninf = system.gammat*state.uninf
    lat = system.betaV*state.vir * state.uninf * system.rho
    act1 = system.alpha*state.lat
    act2 = system.delta*state.uninf
    act = act1+act2
    vir = system.pi*state.act
    
    #FLOW OUT
    uninf -= system.mu*uninf
    lat1 = system.alpha*lat
    lat2 = system.mu*lat1
    act -= system.delta*act
    vir -= system.sigma*vir
    
    return State(uninf = uninf, lat = lat2, act = act, vir = vir)

In [ ]:
def update_func(state, t, system): 
    unpack(system)
    R = state.uninf;
    L = state.lat;
    E = state.act;
    V = state.vir;
    dt = 0.01
    
    rChange = (gammat - mu * R - beta * V * R) * dt
    lChange = (rho * beta * V * R - mu * L - alpha * L) * dt
    eChange = ((1 - rho) * beta * V * R + alpha * L - delta * E) * dt
    vChange = (pi * E - sigma * V) * dt
    
    return State(uninf = rChange, lat = lChange, act = eChange, vir = vChange)

## Run Simulation

In [ ]:
def run_simulation(system, update_func, init):
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.row[t0] = init
    
    for t in linrange(t0, t_end, .01):
        frame.row[t+.01] = update_func(frame.row[t], t, system)
    
    return frame

## Plot Results

In [ ]:
def plot_results(uinf, lat, act, vir):
    plot(uinf, '--', label='Uninfected Cells')
    plot(lat, '-', label='Latent Cells')
    plot(act, '--', label='Uninfected Cells')
    plot(vir, '-', label='Virons')
    decorate(xlabel='Time (days)',
             ylabel='Fraction of Population')

In [ ]:
results = run_simulation(system, update_func, init)
plot_results(results.uninf, results.lat, results.act, results.vir )

In [ ]:
plot(results.vir)